In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab


## TWEETSENT

In [47]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [48]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [49]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [50]:
text_Tweetsent = df_tweet['text']
polarity = np.asarray(df_tweet['sentiment'])

In [51]:
X_text = count_vect.transform(text_Tweetsent)

X_text_transform = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_text_transform.shape

(11533, 25852)

In [52]:
X_train1, X_test_TWEET, Y_train1, Y_test_TWEET = train_test_split(X_text_transform, polarity, test_size=0.3)

In [53]:
X_train12, X_test_TWEET1, Y_train12, Y_test_TWEET1 = train_test_split(df_tweet['text'], df_tweet['sentiment'] , test_size=0.3)

## TASH

In [54]:
df = pd.read_csv("tash-pt.csv")
df = df.dropna()
df = df.drop(columns=['id_twitter'])


In [55]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [56]:
text_TASH = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

In [57]:
X_train2, X_test_TASH, Y_train2, Y_test_TASH = train_test_split(X_text_TASH_, sentiment, test_size=0.3)

In [58]:
X_train21, X_test_TASH21, Y_train21, Y_test_TASH21 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## KANSOAN

In [59]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [60]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [61]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

In [62]:
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [63]:
X_train31, X_test_KANSOAN1, Y_train31, Y_test_KANSOAN1 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## TEST

In [39]:
df = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [40]:
df_all = df_all.dropna()

In [41]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [42]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [43]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [44]:
count_vect = CountVectorizer()

Tweet = count_vect.fit_transform(Tweet)

tfidf_transformer = TfidfTransformer()
X_train_transform = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_transform.shape

(12626, 25852)

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)



In [46]:
X_train_text, X_test_text, Y_train_text, Y_test_text = train_test_split(df_all['Tweet'] , df_all['Polaridade'], test_size=0.3)


## NB - TRAIN -> UNILEX - TEST -> TWEETSENT

In [26]:
clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.45      0.46      0.46      1037
           0       0.34      0.39      0.37       875
           1       0.59      0.53      0.56      1548

    accuracy                           0.48      3460
   macro avg       0.46      0.46      0.46      3460
weighted avg       0.49      0.48      0.48      3460



## NB - TRAIN -> UNILEX - TEST -> TASH

In [27]:
clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.39      0.60      0.47       252
           0       0.47      0.27      0.34       301
           1       0.48      0.48      0.48       284

    accuracy                           0.44       837
   macro avg       0.45      0.45      0.43       837
weighted avg       0.45      0.44      0.43       837



## NB - TRAIN -> UNILEX - TEST -> KANSOAN

In [28]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [29]:
clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.94      0.75      0.84      1378
         0.0       0.00      1.00      0.00         0
         1.0       0.87      0.89      0.88      1390

    accuracy                           0.82      2768
   macro avg       0.60      0.88      0.57      2768
weighted avg       0.90      0.82      0.86      2768



## SVM - TRAIN->UNILEX - TEST->TWEETSENT

In [30]:
clf = SVC().fit(X_train, Y_train) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.50      0.28      0.36      1037
           0       0.32      0.49      0.38       875
           1       0.56      0.56      0.56      1548

    accuracy                           0.46      3460
   macro avg       0.46      0.44      0.43      3460
weighted avg       0.48      0.46      0.45      3460



## SVM T U T TASH

In [31]:
clf = SVC().fit(X_train, Y_train) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.38      0.40       252
           0       0.45      0.36      0.40       301
           1       0.42      0.54      0.47       284

    accuracy                           0.43       837
   macro avg       0.43      0.43      0.43       837
weighted avg       0.43      0.43      0.43       837



## SVM T U T KANSOAN

In [32]:
clf = SVC().fit(X_train, Y_train) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.96      0.57      0.72      1378
         0.0       0.00      1.00      0.00         0
         1.0       0.82      0.91      0.86      1390

    accuracy                           0.74      2768
   macro avg       0.59      0.83      0.53      2768
weighted avg       0.89      0.74      0.79      2768



## RL T U T TWEETSENT

In [33]:
clf = LogisticRegression(max_iter=1000).fit(X_train, Y_train) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.32      0.38      1037
           0       0.34      0.41      0.37       875
           1       0.56      0.61      0.58      1548

    accuracy                           0.47      3460
   macro avg       0.46      0.45      0.45      3460
weighted avg       0.48      0.47      0.47      3460



## RL T U T TASH

In [34]:
clf = LogisticRegression(max_iter=1000).fit(X_train, Y_train) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.43      0.42       252
           0       0.45      0.29      0.35       301
           1       0.42      0.56      0.48       284

    accuracy                           0.42       837
   macro avg       0.43      0.43      0.42       837
weighted avg       0.43      0.42      0.42       837



## RL T U T KANSOAN

In [35]:
clf = LogisticRegression(max_iter=1000).fit(X_train, Y_train) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.97      0.64      0.77      1378
         0.0       0.00      1.00      0.00         0
         1.0       0.82      0.93      0.87      1390

    accuracy                           0.79      2768
   macro avg       0.60      0.86      0.55      2768
weighted avg       0.89      0.79      0.82      2768



## TOTALMENT.C T U T TWEETSENT

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

one = OneHotEncoder(sparse=False)

y_one = one.fit_transform(Y_train.reshape(-1,1))
y_one_test = one.transform(Y_test_TWEET.reshape(-1,1))

fit = model.fit(X_train.todense(), y_one, epochs=10, validation_data=(X_test_TWEET.todense(), y_one_test))

Epoch 1/10
277/277 [==============================] - 3s 11ms/step - loss: 1.0206 - accuracy: 0.4958 - val_loss: 1.0507 - val_accuracy: 0.4587
Epoch 2/10
277/277 [==============================] - 2s 8ms/step - loss: 0.6501 - accuracy: 0.7522 - val_loss: 1.1359 - val_accuracy: 0.4965
Epoch 3/10
277/277 [==============================] - 2s 8ms/step - loss: 0.3104 - accuracy: 0.9016 - val_loss: 1.4872 - val_accuracy: 0.4922
Epoch 4/10
277/277 [==============================] - 2s 8ms/step - loss: 0.1604 - accuracy: 0.9536 - val_loss: 1.8188 - val_accuracy: 0.4905
Epoch 5/10
277/277 [==============================] - 2s 8ms/step - loss: 0.1011 - accuracy: 0.9737 - val_loss: 2.1088 - val_accuracy: 0.4858
Epoch 6/10
277/277 [==============================] - 2s 8ms/step - loss: 0.0719 - accuracy: 0.9809 - val_loss: 2.3175 - val_accuracy: 0.4818
Epoch 7/10
277/277 [==============================] - 2s 8ms/step - loss: 0.0592 - accuracy: 0.9850 - val_loss: 2.4075 - val_accuracy: 0.4855
Epoch

In [37]:
predicted = model.predict(X_test_TWEET.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.48      0.37      0.42      1037
           1       0.35      0.37      0.36       875
           2       0.55      0.62      0.59      1548

    accuracy                           0.48      3460
   macro avg       0.46      0.45      0.46      3460
weighted avg       0.48      0.48      0.48      3460



## KANSOAN

In [66]:
predicted = model.predict(X_test_KANSOAN.todense())

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.94      0.72      0.81      1345
           1       0.38      0.05      0.09      1423
           2       0.00      0.00      0.00         0

    accuracy                           0.38      2768
   macro avg       0.44      0.26      0.30      2768
weighted avg       0.65      0.38      0.44      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [67]:
predicted = model.predict(X_test_TASH.todense())

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.43      0.49      0.46       264
           1       0.42      0.22      0.28       316
           2       0.39      0.56      0.46       257

    accuracy                           0.41       837
   macro avg       0.41      0.42      0.40       837
weighted avg       0.41      0.41      0.39       837



## LSTM T U T TWEETSENT

In [68]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df_all['Tweet'])))

vocab = set_array(df_all['Tweet'])

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [69]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train_text).reshape(-1,1))
y_one_test = one.transform(np.asarray(Y_test_TWEET1).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train_text)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
70/70 [==============================] - 8s 116ms/step - loss: 1.0802 - accuracy: 0.4446 - val_loss: 1.1141 - val_accuracy: 0.2928
Epoch 2/10
70/70 [==============================] - 7s 97ms/step - loss: 0.9473 - accuracy: 0.5552 - val_loss: 1.1253 - val_accuracy: 0.3286
Epoch 3/10
70/70 [==============================] - 6s 93ms/step - loss: 0.7745 - accuracy: 0.6677 - val_loss: 1.1886 - val_accuracy: 0.3893
Epoch 4/10
70/70 [==============================] - 7s 93ms/step - loss: 0.6176 - accuracy: 0.7673 - val_loss: 1.2055 - val_accuracy: 0.4306
Epoch 5/10
70/70 [==============================] - 6s 88ms/step - loss: 0.4882 - accuracy: 0.8324 - val_loss: 1.3371 - val_accuracy: 0.4324
Epoch 6/10
70/70 [==============================] - 6s 89ms/step - loss: 0.4034 - accuracy: 0.8685 - val_loss: 1.4222 - val_accuracy: 0.4208
Epoch 7/10
70/70 [==============================] - 6s 92ms/step - loss: 0.3466 - accuracy: 0.8887 - val_loss: 1.5620 - val_accuracy: 0.4150
Epoch 8/10
7

In [70]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.30      0.34       980
           1       0.32      0.44      0.37       882
           2       0.52      0.49      0.50      1598

    accuracy                           0.42      3460
   macro avg       0.41      0.41      0.40      3460
weighted avg       0.43      0.42      0.42      3460



## KANSOAN

In [71]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.68      0.31      0.43      1338
           1       0.49      0.26      0.34      1430
           2       0.00      0.00      0.00         0

    accuracy                           0.29      2768
   macro avg       0.39      0.19      0.26      2768
weighted avg       0.58      0.29      0.38      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [72]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.36      0.39      0.38       251
           1       0.37      0.22      0.27       317
           2       0.32      0.45      0.37       269

    accuracy                           0.34       837
   macro avg       0.35      0.35      0.34       837
weighted avg       0.35      0.34      0.34       837



## CONV1D T U T TWEET

In [73]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train_text).reshape(-1,1))
y_one_test = one.transform(np.asarray(Y_test_TWEET1).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train_text)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
70/70 [==============================] - 2s 24ms/step - loss: 1.0799 - accuracy: 0.4056 - val_loss: 1.1764 - val_accuracy: 0.2789
Epoch 2/10
70/70 [==============================] - 1s 20ms/step - loss: 0.9373 - accuracy: 0.5655 - val_loss: 1.1236 - val_accuracy: 0.3783
Epoch 3/10
70/70 [==============================] - 1s 20ms/step - loss: 0.6445 - accuracy: 0.7532 - val_loss: 1.1706 - val_accuracy: 0.4289
Epoch 4/10
70/70 [==============================] - 2s 22ms/step - loss: 0.3805 - accuracy: 0.8650 - val_loss: 1.3575 - val_accuracy: 0.4171
Epoch 5/10
70/70 [==============================] - 1s 20ms/step - loss: 0.2430 - accuracy: 0.9167 - val_loss: 1.5881 - val_accuracy: 0.4107
Epoch 6/10
70/70 [==============================] - 1s 20ms/step - loss: 0.1611 - accuracy: 0.9441 - val_loss: 1.7444 - val_accuracy: 0.4173
Epoch 7/10
70/70 [==============================] - 2s 24ms/step - loss: 0.1209 - accuracy: 0.9572 - val_loss: 1.8508 - val_accuracy: 0.4523
Epoch 8/10
70

In [74]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.37      0.38       980
           1       0.32      0.39      0.35       882
           2       0.54      0.49      0.51      1598

    accuracy                           0.43      3460
   macro avg       0.42      0.42      0.42      3460
weighted avg       0.44      0.43      0.44      3460



## KANSOAN 

In [75]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.71      0.60      0.65      1338
           1       0.62      0.28      0.38      1430
           2       0.00      0.00      0.00         0

    accuracy                           0.43      2768
   macro avg       0.44      0.29      0.34      2768
weighted avg       0.66      0.43      0.51      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [76]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.37      0.48      0.42       251
           1       0.42      0.28      0.34       317
           2       0.35      0.40      0.37       269

    accuracy                           0.38       837
   macro avg       0.38      0.39      0.38       837
weighted avg       0.39      0.38      0.37       837



## BDR T U T TWEET

In [86]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train_text).reshape(-1,1))
y_one_test = one.transform(np.asarray(Y_test_TWEET1).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train_text)), y_one, epochs=10, batch_size=32 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
277/277 [==============================] - 6s 22ms/step - loss: 1.0932 - accuracy: 0.3712 - val_loss: 1.1212 - val_accuracy: 0.2526
Epoch 2/10
277/277 [==============================] - 4s 15ms/step - loss: 1.0442 - accuracy: 0.4893 - val_loss: 1.0969 - val_accuracy: 0.3075
Epoch 3/10
277/277 [==============================] - 4s 16ms/step - loss: 0.9407 - accuracy: 0.5516 - val_loss: 1.1093 - val_accuracy: 0.3171
Epoch 4/10
277/277 [==============================] - 5s 17ms/step - loss: 0.8595 - accuracy: 0.5898 - val_loss: 1.1147 - val_accuracy: 0.3168
Epoch 5/10
277/277 [==============================] - 4s 15ms/step - loss: 0.7848 - accuracy: 0.6215 - val_loss: 1.1393 - val_accuracy: 0.3217
Epoch 6/10
277/277 [==============================] - 4s 15ms/step - loss: 0.7145 - accuracy: 0.6520 - val_loss: 1.1702 - val_accuracy: 0.3671
Epoch 7/10
277/277 [==============================] - 4s 15ms/step - loss: 0.6435 - accuracy: 0.7090 - val_loss: 1.1979 - val_accuracy: 0.3861

In [87]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.33      0.36      1009
           1       0.31      0.34      0.32       874
           2       0.48      0.51      0.49      1577

    accuracy                           0.41      3460
   macro avg       0.40      0.39      0.39      3460
weighted avg       0.41      0.41      0.41      3460



## KANSOAN

In [77]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.71      0.60      0.65      1338
           1       0.62      0.28      0.38      1430
           2       0.00      0.00      0.00         0

    accuracy                           0.43      2768
   macro avg       0.44      0.29      0.34      2768
weighted avg       0.66      0.43      0.51      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [79]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.37      0.48      0.42       251
           1       0.42      0.28      0.34       317
           2       0.35      0.40      0.37       269

    accuracy                           0.38       837
   macro avg       0.38      0.39      0.38       837
weighted avg       0.39      0.38      0.37       837

